In [ ]:
import pandas as pd

df = pd.read_csv('Score_Results.csv')

df

,epochs,batch_size,learning_rate,lambda_weight,loss_function,ari,hvg,asw,f1,nmi,sil,graph,pcr,sil_batch,avg_bio,avg_batch
0,1,640,0.0001,0.2,uniform,0.447608,0.291125,0.670756,0.836957,0.730355,0.582232,0.829793,0.429228,0.835066,0.593172,0.698029
1,1,640,0.0001,0.8,uniform,0.447608,0.289500,0.671533,0.833787,0.730355,0.582787,0.829668,0.429975,0.835227,0.592595,0.698290


: 